Библиотеки

In [2]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from olimp.processing import fft_conv
from olimp.precompensation.nn.dataset.psf_gauss import PsfGaussDataset
from ballfish.distribution import DistributionParams
import os
import numpy as np


Наложение шума и фрт на изображениями

In [13]:

def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert("L")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32)
    ])
    return transform(img).squeeze(0)

def prepare_kernel(kernel):
    """Подготовка ядра для FFT свертки"""
    if kernel.dim() == 3:
        kernel = kernel.squeeze(0)
    kernel = torch.fft.fftshift(kernel)
    return kernel / kernel.sum()

def add_noise(image, std):
    """Добавление гауссовского шума"""
    noise = torch.randn_like(image) * std
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0, 1)

# Параметры
noise_levels = [0.01, 0.05, 0.1]
image_files = ['4.1.08.tiff', '5.1.12.tiff', 'house.tiff', 'ruler.512.tiff']

output_dir = "pic"  # Замените на нужный путь

os.makedirs(output_dir, exist_ok=True)

for tiff_file in image_files:
    try:
        png_file = os.path.splitext(tiff_file)[0] + ".png"  # Формируем имя PNG файла
        Image.open(tiff_file).save(os.path.join(output_dir, png_file), "png")
        print(f"Преобразовано: {tiff_file} -> {png_file}")
    except FileNotFoundError:
        print(f"Файл не найден: {tiff_file}")
    except Exception as e:
        print(f"Ошибка при преобразовании {tiff_file}: {e}")
num_frts = 4
output_dir_1 = ['выходные_данные_1', 'выходные_данные_2', 'выходные_данные_3', 'выходные_данные_4']

for i in output_dir_1:
    os.makedirs(i, exist_ok=True)

# Фиксируем seed для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

trio_count = 0
n = -1
"""
for img_file in image_files:
    print(f"\nОбработка изображения: {img_file}")
    img_path = os.path.join("data", img_file)
    img_tensor = load_and_preprocess_image(img_path)
    n = n + 1
    print(n)
    height, width = img_tensor.shape
    output_dir = output_dir_1[n]
    
    
    # Параметры для ФРТ
    center_x_params = {
        "name": "uniform", 
        "a": width/2 - width*0.1,
        "b": width/2 + width*0.1
    }
    center_y_params = {
        "name": "uniform", 
        "a": height/2 - height*0.1,
        "b": height/2 + height*0.1
    }
    theta_params = {
        "name": "uniform", 
        "a": 0.0,
        "b": 360.0
    }
    sigma_x_params = {
        "name": "uniform", 
        "a": 3.0,
        "b": 10.0
    }
    sigma_y_params = {
        "name": "uniform", 
        "a": 3.0,
        "b": 10.0
    }
    
    psf_dataset = PsfGaussDataset(
        width=width,
        height=height,
        center_x=center_x_params,
        center_y=center_y_params,
        theta=theta_params,
        sigma_x=sigma_x_params,
        sigma_y=sigma_y_params,
        size=num_frts,
        seed=42  # Фиксированный seed
    )
    
    for i in range(num_frts):
        print(f"\nГенерация ФРТ {i+1} для {img_file}")
        kernel = psf_dataset[i]
        kernel = prepare_kernel(kernel)
        
        blurred_img = fft_conv(img_tensor, kernel)
        
        for noise_std in noise_levels:
            trio_count += 1
            print(f"Добавление шума со стандартным отклонением {noise_std}")
            noisy_blurred_img = add_noise(blurred_img, noise_std)
            
            plt.figure(figsize=(20, 6))
            
            plt.subplot(131)
            plt.imshow(img_tensor.numpy(), cmap='gray', vmin=0, vmax=1)
            plt.title('Исходное изображение')
            plt.axis('off')
            
            plt.subplot(132)
            kernel_vis = kernel.numpy()
            kernel_vis = (kernel_vis - kernel_vis.min()) / (kernel_vis.max() - kernel_vis.min())
            plt.imshow(kernel_vis, cmap='gray')
            plt.title(f'ФРТ {i+1}')
            plt.axis('off')
            
            plt.subplot(133)
            plt.imshow(noisy_blurred_img.numpy(), cmap='gray', vmin=0, vmax=1)
            plt.title(f'Размытое изображение + шум (σ={noise_std})')
            plt.axis('off')
            
            plt.tight_layout()
            
            # Сохраняем и показываем
            
            output_path = os.path.join(
                output_dir, 
                f"{os.path.splitext(img_file)[0]}_frt{i+1}_noise{noise_std}.png"
            )
            plt.savefig(output_path, bbox_inches='tight', dpi=150)
            #plt.show()
            plt.close()
            
            # Сохраняем тензоры
            torch.save({
                'original': img_tensor,
                'psf': kernel,
                'blurred_noisy': noisy_blurred_img,
                'noise_std': noise_std
            }, output_path.replace('.png', '.pt'))

            
print(f"\nСгенерировано {trio_count} троек изображений")
"""

Преобразовано: 4.1.08.tiff -> 4.1.08.png
Преобразовано: 5.1.12.tiff -> 5.1.12.png
Преобразовано: house.tiff -> house.png
Преобразовано: ruler.512.tiff -> ruler.512.png


'\nfor img_file in image_files:\n    print(f"\nОбработка изображения: {img_file}")\n    img_path = os.path.join("data", img_file)\n    img_tensor = load_and_preprocess_image(img_path)\n    n = n + 1\n    print(n)\n    height, width = img_tensor.shape\n    output_dir = output_dir_1[n]\n\n\n    # Параметры для ФРТ\n    center_x_params = {\n        "name": "uniform", \n        "a": width/2 - width*0.1,\n        "b": width/2 + width*0.1\n    }\n    center_y_params = {\n        "name": "uniform", \n        "a": height/2 - height*0.1,\n        "b": height/2 + height*0.1\n    }\n    theta_params = {\n        "name": "uniform", \n        "a": 0.0,\n        "b": 360.0\n    }\n    sigma_x_params = {\n        "name": "uniform", \n        "a": 3.0,\n        "b": 10.0\n    }\n    sigma_y_params = {\n        "name": "uniform", \n        "a": 3.0,\n        "b": 10.0\n    }\n\n    psf_dataset = PsfGaussDataset(\n        width=width,\n        height=height,\n        center_x=center_x_params,\n        c